In [2]:
!pwd

/home/mln/GIT/0_EXPLORE/github/astronomer-codespaces-test/data


In [3]:
import pandas as pd


df = pd.read_parquet('2024-06-02.parquet')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124950 entries, 0 to 124949
Data columns (total 16 columns):
 #   Column                       Non-Null Count   Dtype                                
---  ------                       --------------   -----                                
 0   id                           124950 non-null  float64                              
 1   record_timestamp             124950 non-null  datetime64[us, pytz.FixedOffset(120)]
 2   stationcode                  124950 non-null  object                               
 3   ebike                        124950 non-null  int64                                
 4   mechanical                   124950 non-null  int64                                
 5   duedate                      124950 non-null  object                               
 6   numbikesavailable            124950 non-null  int64                                
 7   numdocksavailable            124950 non-null  int64                                

In [2]:
import pandas as pd


df = pd.read_parquet('2024-07-24.parquet')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 942480 entries, 0 to 942479
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype                                
---  ------             --------------   -----                                
 0   id                 942480 non-null  int64                                
 1   record_timestamp   942480 non-null  datetime64[us, pytz.FixedOffset(120)]
 2   stationcode        942480 non-null  object                               
 3   ebike              942480 non-null  int64                                
 4   mechanical         942480 non-null  int64                                
 5   duedate            942480 non-null  object                               
 6   numbikesavailable  942480 non-null  int64                                
 7   numdocksavailable  942480 non-null  int64                                
 8   capacity           942480 non-null  int64                                
 9   is_renting     

In [4]:
import duckdb



df = pd.read_parquet('merged_data.parquet')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46011157 entries, 0 to 46011156
Data columns (total 16 columns):
 #   Column                       Dtype                  
---  ------                       -----                  
 0   id                           float64                
 1   record_timestamp             datetime64[us, Etc/UTC]
 2   stationcode                  object                 
 3   ebike                        int64                  
 4   mechanical                   int64                  
 5   duedate                      object                 
 6   numbikesavailable            int64                  
 7   numdocksavailable            int64                  
 8   capacity                     int64                  
 9   is_renting                   object                 
 10  is_installed                 object                 
 11  is_returning                 object                 
 12  name                         object                 
 13  latitude  